<a href="https://colab.research.google.com/github/stavis1/GST_colloquium/blob/main/GST_colloquium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GST Colloquium on Python coding

In [ ]:
#this allows us to interact with google drive from within python
from google.colab import drive
#we mount our google drive which means that we make it acessable as a hard drive
#in our file system under the path '/content/drive'
drive.mount('/content/drive')

In [ ]:
#colab comes with most common packages installed already but we want to use a
#few uncommon ones so we install these ourselves.
!pip install brain-isotopic-distribution
!pip install multiprocesspandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.6/126.6 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.0 MB/s eta 0:00:00


In [ ]:
#it is generally recommended best practice to import the packages that come with
#python first before importing third party packages.
from functools import cache
from multiprocessing import Pool
from collections import defaultdict

from brainpy import isotopic_variants
from numba import jit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import multiprocesspandas
from scipy.optimize import minimize
from sortedcontainers import SortedList

##Data Structures

###Hashmaps

###Search trees

###Example 1: fuzzy matching

###Example 2: file parsing

###Example 3: isotope abundance calculations

##Pandas and Numpy

###Faster .xlsx import

##Minimizers

###Example 1: isotopic packet fitting

##Multiprocessing

###Numba

###Example 1: custom correlation matrix

###multiprocessing.Pool

###Example 2: bootstraping